In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier


In [3]:
df = pd.read_csv('Titanic-Dataset.csv')

In [4]:
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

# Let's make a roadmap to use pipeline

In [5]:
# impute missing values --> do OHE in 'sex' and 'embarked' --> scaling krenge --> feature selection --> decisionTree --> then, we get output 

In [6]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [7]:
df.head()

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S

In [8]:
# Step 1  -> do train_test_split 

X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [9]:
X_train.shape

(712, 7)

In [10]:
y_train.shape

(712,)

In [11]:
# now, do imputation transformer 

trf1 = ColumnTransformer([
         ('impute_age', SimpleImputer(), [2]), # 2 --> 'age' column
         ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])  # 6 --> 'embarked' column 
], remainder='passthrough')

# upto this step, we have only 7 columns

In [12]:
# now, do OneHotEncoding 

trf2 = ColumnTransformer([
         ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,6])   # 1 -> 'sex' column and 6 -> 'embarked' column 
], remainder='passthrough') 

# upto this step, we get total 10 columns(5 col. + 2('sex') col. + 3('embarked') col.)

In [13]:
# for scaling

trf3 = ColumnTransformer([
        ('scale', MinMaxScaler(), slice(0,10))  # for feature scaling, we use here 'MinMaxScaler()' instead of the 'StandardScaler()'  
]) 

In [14]:
# for feature selection 

trf4 = SelectKBest(score_func=chi2, k=8)  # k=8 means that you are using only top 8 columns among 10 columns

In [15]:
# for train the model

trf5 = DecisionTreeClassifier()

# Create Pipeline

In [16]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

In [17]:
# you can also use 'make_pipeline' in place of 'Pipeline'

# alternate syntax

pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)


In [18]:
# train the model

pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000028134325B40>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [19]:
# display pipeline 

from sklearn import set_config
set_config (display='diagram')

In [20]:
# through this you can explore what's happening inside the pipeline 

pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x0000028134325B40>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [21]:
# this gives the mean of 'trf1' SimpleImputer() for 'impute_age'

pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.49884615])

In [22]:
# this gives the mean of 'trf1' SimpleImputer() for 'impute_embarked'

pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [23]:
# now, comes to the prediction

y_pred = pipe.predict(X_test)
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [24]:
# now, check the accuracy 

from sklearn.metrics import accuracy_score 
accuracy_score(y_test,y_pred)

0.6256983240223464